In [1]:
import os
import datetime
import sqlite3
import pandas as pd
import tiktoken

from dotenv import load_dotenv
from openai import OpenAI
from langchain_core.prompts import PromptTemplate

## Config 

In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PATH_TO_DB = os.getenv("PATH_TO_DB")

encoding = tiktoken.get_encoding("cl100k_base")
db_connect = sqlite3.connect(PATH_TO_DB)

In [3]:
def get_llm_results(data):
    results = {
        "created": datetime.datetime.fromtimestamp(answer.created),
        "model": answer.model,
        "total_tokens": answer.usage.total_tokens,
        "prompt_tokens": answer.usage.prompt_tokens,
        "completion_tokens": answer.usage.completion_tokens,
        "answer": answer.choices[0].message.content
    }

    print(f"created: {results['created']}, "
          f"model: {results['model']}, "
          f"total_tokens: {results['total_tokens']}, "
          f"prompt_tokens: {results['prompt_tokens']}, "
          f"completion_tokens: {results['completion_tokens']}\n"
          f"{results['answer']}"
          )

    return results

In [4]:
def save_llm_results(data: dict = None, db: str = PATH_TO_DB):
    try:
        connect = sqlite3.connect(db)
        cursor = connect.cursor()

        cursor.execute(
            "INSERT INTO llm_results(created, model, total_tokens, prompt_tokens, completion_tokens, prompt, answer) VALUES (?,?,?,?,?,?,?)",
            (data["created"], data["model"], data["total_tokens"], data["prompt_tokens"],
             data["completion_tokens"], prompt, data["answer"])
        )
        connect.commit()
        connect.close()

        print(f"llm_results saved to database")

    except Exception as e:
        print(e)

## Load data

In [5]:
interviews = {}
files = [file for file in os.listdir("../data/raw/") if file.endswith(".txt")]
for file in files:
    with open(f"../data/raw/{file}", "r") as f:
        interviews[file.split(".")[0]] = f.read()

interviews = dict(sorted(interviews.items()))

In [6]:
for key, val in interviews.items():
    print(f"{key}, len: {len(val)}, tokens: {len(encoding.encode(val))}")

interview_1, len: 36019, tokens: 16647
interview_2, len: 33945, tokens: 15517
interview_3, len: 37991, tokens: 17444
interview_4, len: 39345, tokens: 17579
interview_5, len: 50578, tokens: 23353


In [7]:
roles = pd.read_sql(sql="select * from prompts_roles", con=db_connect)
roles

,id,role,instruction
0,1,You are (acting as) a researcher in the field ...,None
1,2,Act as an outstanding professional social scie...,None
2,3,YOU ARE A LEADING EXPERT IN QUALITATIVE ANALYS...,\nKEY OBJECTIVES:\n- THOROUGHLY REVIEW TRANSCR...


In [16]:
roles.iloc[0]["role"]

'You are (acting as) a researcher in the field of quality (for example, Michael Quinn Patton, Norman K. Denzin, John W. Creswell). You have worked for Google and Nielsen Norman Group for decades. You know all about qualitative research methods and are able to apply them when analyzing interviews with users.'

## Prompt

In [9]:
role_system = "Выступайте в роли выдающегося профессионального социолога, специализирующегося на социологии и качественных исследованиях, знакомого со всеми классическими и современными исследовательскими практиками. Ваш образец для подражания - социолог Свен Бринкманн. Вы в полной мере владеете всеми теоретическими и практическими навыками и передовым опытом, накопленным в области социологии, и творчески и гибко применяете их в соответствии с поставленными мной задачами. Не стесняйтесь смешивать и сочетать различные исследовательские методы и технички, при этом оставаясь в рамках научного метода."
role_user = interviews["interview_1"]
role_assistant = """
1. Выполнение первоначального кодирования (также известного как открытое кодирование):
    - Внимательно прочитайте данную расшифровку и выделите или подчеркните ключевые фразы, слова или предложения, которые отражают важные идеи, концепции или опыт.
    - Присвойте этим выделенным сегментам предварительные коды или обозначения, используя короткие описательные фразы или слова, которые суммируют основную идею.
    - Ведите постоянный список созданных вами кодов и последовательно применяйте их к тексту.
    
Я хочу, чтобы коды были подробными и описательными. Я хочу, чтобы вы применили коды к предложениям или частям предложений, а позже, когда вы составите список кодов, я хочу, чтобы вы могли сообщить мне, к каким предложениям или частям предложений были применены эти коды. Другими словами, когда я прошу вас предоставить мне примеры цитат для создаваемых вами кодов, я бы хотел, чтобы вы могли это сделать.

2. Поиск тем:
    - Просмотрите исходные коды, которые вы назначили, и найдите среди них шаблоны, сходства или взаимосвязи.
    - Сгруппируйте связанные коды в более широкие категории или темы, которые охватывают основные темы, обсуждаемые в данной расшифровке.
    - Определите любые подтемы в рамках каждой основной темы, чтобы обеспечить более детальное понимание тем.

3. Сгруппируйте проблемы по категориям:
    - Просмотрите список проблем и найдите среди них общие черты.
    - Разработайте набор категорий или типов проблем, которые могут охватывать различные упомянутые проблемы.
    - Отнесите каждую проблему к одной или нескольким соответствующим категориям.
    - Составьте сводную таблицу или матрицу, показывающую проблемы, сгруппированные по категориям.

4. Сделайте выводы и сформулируйте гипотезы:
    - Поразмышляйте над темами, частотой возникновения проблем и категориями проблем, которые вы определили.
    - Подумайте, что эти результаты говорят об опыте, перспективах или трудностях респондентов.
    - Ищите в данных любые неожиданные закономерности, связи или контрасты.
    - Разрабатывайте предварительные объяснения или гипотезы о том, почему возникли определенные темы или проблемы и как они могут быть связаны друг с другом или с более широкими контекстуальными факторами.
    - Рассмотрите альтернативные объяснения и контрпримеры, чтобы уточнить свои гипотезы.
    - Определите области, в которых могут потребоваться дополнительные исследования или анализ для проверки или подтверждения ваших идей.
"""

In [10]:
template = """
"System": {role_system}\n\n 
"User": {role_user}\n\n
"Assistant": {role_assistant}\n\n

Summary:
"""

template_prompt = PromptTemplate(
    template=template,
    input_variables=["role_system", "role_user", "role_assistant"]
)

prompt = template_prompt.format(role_system=role_system, role_user=role_user, role_assistant=role_assistant)

In [11]:
len(encoding.encode(prompt))

17947

In [12]:
prompt

'\n"System": Выступайте в роли выдающегося профессионального социолога, специализирующегося на социологии и качественных исследованиях, знакомого со всеми классическими и современными исследовательскими практиками. Ваш образец для подражания - социолог Свен Бринкманн. Вы в полной мере владеете всеми теоретическими и практическими навыками и передовым опытом, накопленным в области социологии, и творчески и гибко применяете их в соответствии с поставленными мной задачами. Не стесняйтесь смешивать и сочетать различные исследовательские методы и технички, при этом оставаясь в рамках научного метода.\n\n \n"User": Запись, наши с вами обсуждения.\n\nСмотрите, давайте тогда с самого начала, вот напомним, просто, чтобы у нас человек, который будет расшифровывать, чтобы он не бегал там по разным записям, да?\n\nРасскажите, пожалуйста, еще раз немного о себе.\n\nСколько вам лет, где живете и как давно работаете риэлтором?\n\n42 года, живу в Москве, работаю порядка, ну, мне сейчас сложно сказать,

## OpenAI gpt-4-omni

In [13]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [14]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f"{role_system}"},
        {"role": "user", "content": f"{role_user}"},
        {"role": "assistant", "content": f"{role_assistant}"}
    ]
)

answer = completion

In [15]:
clear_answer = get_llm_results(answer)
save_llm_results(clear_answer)

created: 2024-05-14 16:20:25, model: gpt-4o-2024-05-13, total_tokens: 12186, prompt_tokens: 10629, completion_tokens: 1557
Конечно, давайте вместе рассмотрим процесс анализа данных данного интервью, используя метод открытого кодирования. Начнем с тщательного чтения текста и выделения ключевых фраз и предложений для назначения кодов.

***Кодировка:***

```plaintext
- Интерес к риэлторской деятельности начался с личного опыта: "...продать свою квартиру, поэтому погрузилась в изучение данной процедуры...", "...проделав эту процедуру, потом помогла знакомым..."
  - Код: Личное начало
  - Код: Самостоятельное изучение

- Почему заниматься частным риэлторством: "...началось с того, что нужно было продать свою квартиру, поэтому погрузилась в изучение данной процедуры", "работаю сама по себе, сама на себя, не в агентстве"
  - Код: Независимость
  - Код: Самостоятельность в риэлторстве

- Сфера работы и важные аспекты сделок: "...вторичное жилье... первичное жилье не требует присутствия риэлтор